# test1.ipynb
First test of the delegational learning rule in the context of the spinal cord project.

Let $S_F, S_P, S_{P-F}, M_C$ denote the regions of the cortical model, and $s_F, s_P, s_{P-F}, m_C$ denote their current activities.
Assume that each unit $u_j$ in $M_C$ is associated with a vector $\bar{v}_j$, and that the ouput of the plant is $p = \sum_j u_j \bar{v}_j$. We also set $s_F = p$. The rule should be such that $p \approx s_P$.

The plant $P$ can be implemented as a population of linear units.
If the connection matrix from $M_C$ to $P$ is called $W$, then
$\bar{v}_j = (W_{1,j}, W_{2,j}, ..., W_{n,j})^T \equiv W_{:,j}$. 

In [1]:
from draculab import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Create the neuronal populations
N = 8 # size of each population
net_params = {'min_delay' : 0.01,
              'min_buff_size' : 10 }
SF_params = {'type' : unit_types.sigmoidal,
             'thresh' : 0.,
             'slope' : 1.,
             'init_val' : 0.5,
             'tau' : 0.05 }
SP_params = {'type' : unit_types.source,
             'init_val' : 0.5,
             'function' : lambda x: None }
SPF_params = {'type' : unit_types.sigmoidal,
             'thresh' : 0.,
             'slope' : 1.,
              'init_val' : 0.5,
             'tau' : 0.05 }
P_params = {'type' : unit_types.linear,
            'init_val' : 0.5,
            'tau' : 0.05 }

net = network(net_params)
SF = net.create(N, SF_params)
SP = net.create(N, SP_params)
SPF = net.create(N, SPF_params)
P = net.create(N, P_params)

# Create the connections


In [13]:
from units import *

In [17]:
help(units.linear)

Help on class linear in module units.units:

class linear(unit)
 |  An implementation of a linear unit.
 |  
 |  The output approaches the sum of the inputs multiplied by their synaptic weights,
 |  evolving with time constant 'tau'.
 |  
 |  Method resolution order:
 |      linear
 |      unit
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, ID, params, network)
 |      The unit constructor.
 |      
 |      Args:
 |          ID, params, network: same as in the 'unit' parent class.
 |          In addition, params should have the following entries.
 |              REQUIRED PARAMETERS
 |              'tau' : Time constant of the update dynamics.
 |      
 |      Raises:
 |          AssertionError.
 |  
 |  derivatives(self, y, t)
 |      This function returns the derivatives of the state variables at a given point in time. 
 |      
 |      Args: 
 |          y : a 1-element array or list with the current firing rate.
 |          t: time when the derivative